In [1]:
import cv2
import os
import numpy as np
import sklearn
from sklearn.cluster import MiniBatchKMeans
from scipy.spatial import distance
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import random
from xgboost import XGBClassifier
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/anna/opt/anaconda2/envs/p36workshop/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
def load_images_from_folder(folder):
    images = list()
    labels = list()
    i = 0
    for filename in os.listdir(folder):
        if filename !='.DS_Store':    
            path = folder + "/" + filename
            for cat in os.listdir(path):
                img = cv2.imread(path + "/" + cat,0)
                #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                if img is not None:
                    images.append(img)
                    labels.append(i)
            i = i + 1
    return images, labels

In [2]:
def load_images_class(folder):
    images = list()
    for cat in os.listdir(folder):
        img = cv2.imread(folder + "/" + cat,0)
        if img is not None:
            images.append(img)
    images_train, images_test = sklearn.model_selection.train_test_split(images, test_size = 50, random_state = 42)
    return images_train, images_test

In [3]:
def descriptor_features(X):
    descriptor_list = []
    akaze = cv2.ORB_create()
    for i in range(0, len(X)):
        kp,des = akaze.detectAndCompute(X[i], None)
        descriptor_list.extend(des)
    return descriptor_list

In [4]:
def kmeans(k, descriptor_list):
    kmeans = MiniBatchKMeans(n_clusters=k, n_init=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_
    return visual_words

In [5]:
def find_index(image, center):
    count = 0
    ind = 0
    dist = 0
    for i in range(len(center)):
        if(i == 0):
            count = distance.euclidean(image, center[i])
            dist = count
            #count = L1_dist(image, center[i])
        else:
            dist = distance.euclidean(image, center[i])
            #dist = L1_dist(image, center[i])
        if(dist < count):
            ind = i
            count = dist
    return ind

In [6]:
def image_class(X, centers):
    dict_feature = list()
    akaze = cv2.ORB_create()
    for i in range(0, len(X)):
        #print(i)
        kp,des = akaze.detectAndCompute(X[i], None)
        histogram = np.zeros(len(centers))
        for each_feature in des:
            
            ind = find_index(each_feature, centers)
            histogram[ind] += 1
        dict_feature.append(histogram)
    return dict_feature

In [7]:
X_Train = list()
y_Train = list()
X_Test = list()
y_Test = list()
filenames = ('True_Budda', 'True_parfum', 'False')
for i in range(0, 3):
    x1, x2 = load_images_class('Photos/'+filenames[i])
    X_Train.extend(x1)
    X_Test.extend(x2)
    for j in range(0, len(x1)):
        y_Train.append(i)
    for j in range(0, len(x2)):
        y_Test.append(i)
X_train, y_train = sklearn.utils.shuffle(X_Train, y_Train, random_state = 42)
X_test, y_test = sklearn.utils.shuffle(X_Test, y_Test, random_state = 42)



In [8]:
descriptors = descriptor_features(X_train)
words = kmeans(200, descriptors)
X_train_new = image_class(X_train, words)
X_test_new = image_class(X_test, words)

парфюм = 1, Будда = 0, мусор = 2

### Функція для знаходження статистики по кожному методу 

In [9]:
def accuracy(y1, y2):
    S = 0
    for i in range(0, len(y1)):
        if y1[i]==y2[i]:
            S = S + 1
    S = S/len(y1)
    print("Accuracy: " + str(S))

In [10]:
def conf_matrix(y_test, y_pred):
    matrix = np.zeros((3, 3))
    for i in range(0, len(y_test)):
        a = y_test[i]
        b = y_pred[i]
        matrix[a][b]= matrix[a][b] + 1
    print("Confusion matrix\n")
    print(matrix)

# Вибір класифікатора без підбору параметрів

## Градієнтний бустінг

In [11]:
model = GradientBoostingClassifier()
model.fit(X_train_new, y_train)
y_pred = model.predict(X_test_new)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.5266666666666666
Confusion matrix

[[36. 13.  1.]
 [12. 37.  1.]
 [26. 18.  6.]]


## Випадковий ліс

In [12]:
model = RandomForestClassifier()
model.fit(X_train_new, y_train)
y_pred = model.predict(X_test_new)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.5
Confusion matrix

[[38. 12.  0.]
 [14. 36.  0.]
 [35. 14.  1.]]


## Дерево прийняття рішень

In [13]:
model = DecisionTreeClassifier()
model.fit(X_train_new, y_train)
y_pred = model.predict(X_test_new)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.4266666666666667
Confusion matrix

[[29. 15.  6.]
 [18. 28.  4.]
 [26. 17.  7.]]


## XGBClassifier

In [14]:
model = XGBClassifier()
arr = np.array(y_train)
arr1 = np.array(X_train_new)
arr2 = np.array(X_test_new)
model.fit(arr1, arr)
y_pred = model.predict(arr2)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.52
Confusion matrix

[[32. 14.  4.]
 [ 8. 39.  3.]
 [30. 13.  7.]]


# Підбір параметрів для кожного з методів за допомогою GridSearch

## GradientBoostingClassifier

In [15]:
parameters = {'min_samples_split':(2,4, 6), 
              'min_samples_leaf':(1, 3,4), 
              'max_depth':(3,5,8),
             'n_estimators':(100, 150)}
m = GradientBoostingClassifier()
model1 = GridSearchCV(m, parameters, return_train_score = True)
model1.fit(X_train_new, y_train)


GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'max_depth': (3, 5, 8), 'min_samples_leaf': (1, 3, 4),
                         'min_samples_split': (2, 4, 6),
                         'n_estimators': (100, 150)},
             return_train_score=True)

In [16]:
%%time
y_pred = model1.predict(X_test_new)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.5
Confusion matrix

[[31. 15.  4.]
 [ 8. 39.  3.]
 [26. 19.  5.]]
CPU times: user 5.39 ms, sys: 264 µs, total: 5.66 ms
Wall time: 5.72 ms


In [17]:
model1.best_params_

{'max_depth': 5,
 'min_samples_leaf': 4,
 'min_samples_split': 6,
 'n_estimators': 150}

In [18]:
model1.best_score_

0.6929125138427464

## RandomForestClassifier

In [19]:
parameters = {'n_estimators':(100,125, 150), 
              'min_samples_split':(2, 4, 3),
             'min_samples_leaf':(1, 3, 5)}
m = RandomForestClassifier()
model2 = GridSearchCV(m, parameters, return_train_score = True)
model2.fit(X_train_new, y_train)
y_pred = model2.predict(X_test_new)


In [20]:
%%time
y_pred = model2.predict(X_test_new)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.4866666666666667
Confusion matrix

[[35. 14.  1.]
 [12. 38.  0.]
 [36. 14.  0.]]
CPU times: user 20.6 ms, sys: 2.66 ms, total: 23.2 ms
Wall time: 23.2 ms


In [21]:
model2.best_params_

{'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 125}

In [22]:
model2.best_score_

0.683610188261351

## Decision Tree Classifier

In [23]:
parameters = {'splitter':('best', 'random'),
              'min_samples_split':(2, 4, 6),
              'min_samples_leaf':(1, 3, 4),
              'criterion':('gini', 'entropy')}
m = DecisionTreeClassifier()
model3 = GridSearchCV(m, parameters, return_train_score = True)
model3.fit(X_train_new, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'min_samples_leaf': (1, 3, 4),
                         'min_samples_split': (2, 4, 6),
                         'splitter': ('best', 'random')},
             return_train_score=True)

In [24]:
%%time
y_pred = model3.predict(X_test_new)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.4
Confusion matrix

[[28. 17.  5.]
 [21. 27.  2.]
 [26. 19.  5.]]
CPU times: user 1.85 ms, sys: 699 µs, total: 2.54 ms
Wall time: 2.54 ms


In [25]:
model3.best_params_

{'criterion': 'gini',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'splitter': 'random'}

In [26]:
model3.best_score_

0.6269102990033224

## XGBoost Classifier

In [27]:
parameters = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }
m = XGBClassifier()
model4 = GridSearchCV(m, parameters, return_train_score = True)
arr = np.array(y_train)
arr1 = np.array(X_train_new)
arr2 = np.array(X_test_new)
model4.fit(arr1, arr)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None...
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameters=None,
                                     verbosity=None),
             par

In [28]:
%%time
y_pred = model4.predict(arr2)
accuracy(y_test, y_pred)
conf_matrix(y_test, y_pred)

Accuracy: 0.5666666666666667
Confusion matrix

[[38. 12.  0.]
 [10. 40.  0.]
 [32. 11.  7.]]
CPU times: user 19.2 ms, sys: 3.06 ms, total: 22.3 ms
Wall time: 18.1 ms


In [29]:
model4.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 500,
 'objective': 'reg:squarederror',
 'subsample': 0.7}

In [30]:
model4.best_score_

0.6930232558139535

# Робота з відео

За длпомогою OpenCV обробимо 2 коротеньких відео, на яких є один чи інший об'єкт, застосовуючи до кожного 5го кадру кожну з моделей; бедмо писати поверх кадру в кожний момент, який саме об'єкт був знайдений і чи був знайдений взагалі. В результаті отримуємо 8 відео

In [39]:
def create_video(file_name_read, file_name_write):
    cap1 = cv2.VideoCapture(file_name_read)
    frame_width = int(cap1.get(3))
    frame_height = int(cap1.get(4))
    out = cv2.VideoWriter(file_name_write,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    #set method here
    # and don't forget to use appropriate model
    akaze = cv2.ORB_create()
    # features for text
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (50, 50) 
    fontScale = 1
    color = (255, 0, 0) 
    thickness = 2
    i = 0
    while cap1.isOpened():
        ret, frame = cap1.read() 
        if not ret:
            break
        if i%5 == 0:
            kp,des = akaze.detectAndCompute(frame, None)
            histogram = np.zeros(len(words))
            for each_feature in des:
                ind = find_index(each_feature, words)
                histogram[ind] += 1
            histogram = histogram.reshape(1, -1)
            # трошки криво, тут вручну вказує номер моделі з якою працюємо
            y_pred = model4.predict(histogram)
            if y_pred == 1:
                frame = cv2.putText(frame, 'Parfum', org, font,fontScale, color, thickness, cv2.LINE_AA)
            if y_pred == 0:
                frame = cv2.putText(frame, 'Budda', org, font,fontScale, color, thickness, cv2.LINE_AA)
            if y_pred == 2:
                frame = cv2.putText(frame, 'Nothing', org, font,fontScale, color, thickness, cv2.LINE_AA)
            out.write(frame)
        i = i+1
    cap1.release()
    out.release()
    cv2.destroyAllWindows()

In [34]:
create_video('/Users/anna/detection/Lab3/video.mp4', 'Videos/orb_video1_gradient_boosting.avi')
#create_video('/Users/anna/detection/video2.mp4', 'Videos/orb_video2_gradient_boosting.avi')

In [36]:
create_video('/Users/anna/detection/Lab3/video.mp4', 'Videos/orb_video1_random_forest.avi')
#create_video('/Users/anna/detection/video2.mp4', 'Videos/orb_video2_random_forest.avi')

In [38]:
create_video('/Users/anna/detection/Lab3/video.mp4', 'Videos/orb_video1_decision_tree.avi')
#create_video('/Users/anna/detection/video2.mp4', 'Videos/orb_video2_decision_tree.avi')

In [40]:
create_video('/Users/anna/detection/Lab3/video.mp4', 'Videos/orb_video1_xgboost.avi')
#create_video('/Users/anna/detection/video2.mp4', 'Videos/orb_video2_xgboost.avi')